# Trabalho de C209-L6

Importação das Bibliotecas

In [3]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx
import numpy as np

Carregar o Vídeo

In [4]:
video = VideoFileClip("Trabalho.mp4")



Função para inverter o eixo da imagem

In [5]:
def inverter_eixo(clip, eixo="horizontal"):
    if eixo == "horizontal":
        return clip.fx(vfx.mirror_x)
    elif eixo == "vertical":
        return clip.fx(vfx.mirror_y)
    else:
        raise ValueError("O eixo deve ser 'horizontal' ou 'vertical'.")


Dividir o vídeo em segmentos de 20 segundos para inverter o eixo

In [6]:
duration = video.duration
segments = []
for start in range(0, int(duration), 20):
    end = min(start + 20, duration)
    segment = video.subclip(start, end)
    segment = inverter_eixo(segment, eixo="horizontal" if (start // 20) % 2 == 0 else "vertical")
    segments.append(segment)

Concatenar os segmentos com a inversão do eixo

In [7]:
video_invertido = concatenate_videoclips(segments)

Função para aplicar redução e normalização do volume

In [ ]:
def ajustar_volume_em_blocos(audio_clip, intervalo=30, silencio=10):
    """
    Ajusta o volume em blocos de tempo. Cada bloco tem o volume diminuído
    gradativamente nos últimos `silencio` segundos, e volta ao volume normal
    no início do próximo bloco.
    """
    def modificar_volume(get_frame, t):
        t = np.array(t)  
        bloco = np.floor(t / intervalo).astype(int)  
        inicio_bloco = bloco * intervalo
        fim_bloco = inicio_bloco + intervalo
        fim_reducao = fim_bloco - silencio

        volume = np.where(
            t < fim_reducao,  
            1,  
            np.where(
                t < fim_bloco,  
                (fim_bloco - t) / silencio,  
                0  
            )
        )
        return get_frame(t) * volume[:, np.newaxis]

    return audio_clip.fl(modificar_volume)

Ajustar o áudio do vídeo com redução e normalização por blocos

In [9]:
audio_ajustado = ajustar_volume_em_blocos(video_invertido.audio, intervalo=30, silencio=10)
video_com_audio_ajustado = video_invertido.set_audio(audio_ajustado)

Cortar do segundo 60 ao 80

In [10]:
segmento_cortado = video_com_audio_ajustado.subclip(60, 80)


Reorganizar o vídeo final

In [11]:
video_final = concatenate_videoclips([
    video_com_audio_ajustado.subclip(0, 60),
    video_com_audio_ajustado.subclip(80, video_com_audio_ajustado.duration),
    segmento_cortado
])

Exportar o Vídeo Final

In [12]:
video_final.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")


Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
